<a href="https://colab.research.google.com/github/gautamHCSCV/Image-Classification-on-Edges-using-BNN/blob/main/BNN%20on%20images/BNN_on_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import warnings
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense,Conv2D,Flatten, Input,MaxPool2D
from tensorflow.keras.models import Model
warnings.filterwarnings('ignore')

In [ ]:
class Binarize(tf.keras.constraints.Constraint):
  def __init__(self, ref_value = 1):
    self.ref_value = ref_value

  def __call__(self, w):
    return tf.math.sign(w)*self.ref_value

  def get_config(self):
    return {'ref_value': self.ref_value}

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# mnist dataset

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
ds_train.element_spec

(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [ ]:
inp = Input((28,28,1))
x = Conv2D(8,(3,3), activation = 'relu', kernel_constraint=Binarize(), bias_constraint=Binarize())(inp)
x = MaxPool2D(2)(x)
x = Conv2D(16,(3,3), activation = 'relu', kernel_constraint=Binarize(), bias_constraint=Binarize())(x)
x = MaxPool2D(2)(x)
x = Flatten()(x)
out = Dense(10, activation='softmax', kernel_constraint=Binarize(), bias_constraint=Binarize())(x)

model = Model(inputs = inp, outputs = out)
print(model.summary())
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(ds_train,epochs=3,validation_data=ds_test)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                4010

In [ ]:
model.weights

[<tf.Variable 'conv2d_6/kernel:0' shape=(3, 3, 1, 8) dtype=float32, numpy=
 array([[[[ 1.,  1.,  1., -1., -1., -1., -1.,  1.]],
 
         [[-1., -1.,  1., -1., -1.,  1.,  1., -1.]],
 
         [[ 1., -1.,  1., -1., -1., -1.,  1., -1.]]],
 
 
        [[[-1.,  1., -1., -1., -1.,  1.,  1.,  1.]],
 
         [[ 1., -1., -1., -1., -1.,  1., -1., -1.]],
 
         [[-1.,  1.,  1., -1.,  1.,  1.,  1., -1.]]],
 
 
        [[[-1.,  1., -1.,  1.,  1., -1., -1., -1.]],
 
         [[ 1.,  1.,  1.,  1.,  1., -1., -1., -1.]],
 
         [[ 1., -1.,  1., -1., -1.,  1.,  1.,  1.]]]], dtype=float32)>,
 <tf.Variable 'conv2d_6/bias:0' shape=(8,) dtype=float32, numpy=array([-1.,  1.,  1.,  1., -1., -1.,  1., -1.], dtype=float32)>,
 <tf.Variable 'conv2d_7/kernel:0' shape=(3, 3, 8, 16) dtype=float32, numpy=
 array([[[[ 1., -1.,  1., ..., -1., -1., -1.],
          [ 1., -1.,  1., ..., -1.,  1., -1.],
          [ 1.,  1., -1., ..., -1.,  1., -1.],
          ...,
          [-1., -1.,  1., ..., -1.,  1., -1.],

In [ ]:
inp = Input((28,28,1))
x = Conv2D(8,(3,3), activation = 'relu', kernel_constraint=Binarize(), bias_constraint=Binarize())(inp)
x = MaxPool2D(2)(x)
x = Conv2D(16,(3,3), activation = 'relu', kernel_constraint=Binarize(), bias_constraint=Binarize())(x)
x = MaxPool2D(2)(x)
x = Flatten()(x)
out = Dense(10, activation='softmax')(x)

model = Model(inputs = inp, outputs = out)
print(model.summary())
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(ds_train,epochs=3,validation_data=ds_test)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                4010

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME, kernel_constraint=Binarize(), bias_constraint=Binarize())
        self.bn_1 = BatchNormalization(beta_constraint=Binarize(), gamma_constraint=Binarize())
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME, kernel_constraint=Binarize(), bias_constraint=Binarize())
        self.bn_2 = BatchNormalization(beta_constraint=Binarize(), gamma_constraint=Binarize())
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same", kernel_constraint=Binarize(), bias_constraint=Binarize())
            self.res_bn = BatchNormalization(beta_constraint=Binarize(), gamma_constraint=Binarize())

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out

class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal", kernel_constraint=Binarize(), bias_constraint=Binarize())
        self.init_bn = BatchNormalization(beta_constraint=Binarize(), gamma_constraint=Binarize())
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

model = ResNet18(10)
model.build(input_shape = (None,28,28,1))
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "res_net18_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_108 (Conv2D)          multiple                  3200      
_________________________________________________________________
batch_normalization_100 (Bat multiple                  256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling multiple                  0         
_________________________________________________________________
resnet_block_40 (ResnetBlock multiple                  74368     
_________________________________________________________________
resnet_block_41 (ResnetBlock multiple                  74368     
_________________________________________________________________
resnet_block_42 (ResnetBlock multiple             